In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_absolute_error as mae

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
xgb_subm = pd.read_csv("submission_10fold-average-xgb_fairobj_1125.883448_2016-12-02-20-24.csv")
nn_subm = pd.read_csv("submission_keras_all_set_2.csv")

In [9]:
xgb_subm.head()

,id,loss
0,4,1547.478027
1,6,1979.358032
2,9,9984.505859
3,12,6522.488770
4,15,783.580444


In [10]:
nn_subm.head(10)

,id,loss
0,4,3237.365588
1,6,3528.164941
2,9,17419.277246
3,12,15139.904590
4,15,1480.845624
5,17,4702.218115
6,21,3657.809351
7,28,1413.637518
8,32,6065.108545
9,43,5629.851172


In [11]:
merged = xgb_subm.merge(nn_subm, on="id", how="inner")
merged.columns = ["id", "loss_xgb", "loss_nn"]

In [12]:
merged.head()

,id,loss_xgb,loss_nn
0,4,1547.478027,3237.365588
1,6,1979.358032,3528.164941
2,9,9984.505859,17419.277246
3,12,6522.488770,15139.904590
4,15,783.580444,1480.845624


In [33]:
merged["weighted"] = 0.8* merged["loss_xgb"] + 0.2* merged["loss_nn"]

In [34]:
merged.head()

,id,loss_xgb,loss_nn,weighted
0,4,1547.478027,1531.863665,1539.670846
1,6,1979.358032,1762.798763,1871.078398
2,9,9984.505859,8946.321626,9465.413743
3,12,6522.488770,6896.642773,6709.565771
4,15,783.580444,767.032419,775.306432


In [35]:
res = merged[["id", "weighted"]]
res.columns = ["id", "loss"]
res.to_csv("submissions/submission_05xgb_05nn.csv", index=False)

### stack

In [152]:
train = pd.read_csv("../data/train.csv")
oof_xgb = pd.read_csv("oof_df_xgb.csv")
oof_nn = pd.read_csv("preds_oof_NN.csv")
#oof_nn = pd.read_csv("submission_keras_average_oof.csv")
oof_nn_2 = pd.read_csv("preds_oof_NN_2.csv")
preds_xgb = pd.read_csv("submission_10fold-average-xgb_fairobj_1127.609963_2016-12-07-07-38.csv")
preds_nn = pd.read_csv("submission_keras_all_set.csv")
preds_nn_2 = pd.read_csv("submission_keras_all_set_2_susp.csv")

preds_nn_oof_test = pd.read_csv("submission_keras_average_oof.csv")
preds_nn_oof_test_2 = pd.read_csv("submission_keras_average_oof_2.csv")
preds_nn_oof_test_3 = pd.read_csv("submission_keras_average_oof_3.csv")

In [153]:
train = train[["id","loss"]]
preds = preds_xgb.merge(preds_nn, on="id")
preds.columns = ["id", "loss_xgb", "loss_nn"]
preds = preds.merge(preds_nn_2, on="id")
preds.columns = ["id", "loss_xgb", "loss_allset_nn1", "loss_allset_nn_2"]
preds.loss_allset_nn_2 = preds.loss_allset_nn_2/2

In [154]:
preds_nn_test_oof_average = preds_nn_oof_test.merge(preds_nn_oof_test_2, on="id").merge(preds_nn_oof_test_3, on="id")
preds_nn_test_oof_average.columns = ["id", "pred_av_nn1", "pred_av_nn2", "pred_av_nn3"]

In [189]:
preds_nn_test_oof_average.head()

,id,pred_av_nn1,pred_av_nn2,pred_av_nn3,pred_av_nn_mean
0,4,1531.863665,1572.027185,1552.287822,1552.059556
1,6,1762.798763,1788.709243,1783.019814,1778.175939
2,9,8946.321626,9007.457070,8934.108457,8962.629042
3,12,6896.642773,7141.388379,7423.074727,7153.701952
4,15,767.032419,777.997367,757.449355,767.493046


In [200]:
preds_big = preds.merge(preds_nn_test_oof_average)

In [206]:
preds_big.head()

,id,loss_xgb,loss_allset_nn1,loss_allset_nn_2,pred_av_nn1,pred_av_nn2,pred_av_nn3,pred_av_nn_mean
0,4,1512.877930,1571.079736,1618.682794,1531.863665,1572.027185,1552.287822,1569.188240
1,6,1948.610962,1802.400940,1764.082471,1762.798763,1788.709243,1783.019814,1780.202246
2,9,10234.997070,8795.713916,8709.638623,8946.321626,9007.457070,8934.108457,8878.647938
3,12,6416.453125,7165.508398,7569.952295,6896.642773,7141.388379,7423.074727,7239.313314
4,15,792.276978,779.901031,740.422812,767.032419,777.997367,757.449355,764.560597


In [205]:
preds_big["pred_av_nn_mean"] = preds_big.loss_allset_nn1*0.2 + preds_big.loss_allset_nn_2*0.2\
+ preds_big.pred_av_nn1*0.2 + preds_big.pred_av_nn2*0.2+ preds_big.pred_av_nn3*0.2 

In [230]:
preds_big["y_pred_subm"] = (1.01*(0.55*preds_big.loss_xgb + 0.45*preds_big.pred_av_nn_mean) - 10)

In [231]:
res = preds_big[["id", "y_pred_subm"]]
res.columns = ["id", "loss"]
res.to_csv("../submissions/submission_weighted_big_averagenn_055xgb_045nn_shift_17.csv", index=False)


In [193]:
oof_xgb.y_true = oof_xgb.y_true.apply(lambda x: np.exp(x) - 200)
oof_xgb.y_pred = oof_xgb.y_pred.apply(lambda x: np.exp(x) - 200)
oof_xgb.columns = ["id", "loss", "loss_pred_xgb"]
oof_xgb.id = oof_xgb.id.apply(lambda x: int(x))

AttributeError: 'DataFrame' object has no attribute 'y_true'

In [196]:
oof_xgb.head(10)

,id,loss,loss_pred_xgb
0,1,2213.18,1958.702840
1,2,1283.60,1766.157787
2,5,3005.09,4465.905258
3,10,939.85,1054.908671
4,11,2763.85,3282.222525
5,13,5142.87,3791.854788
6,14,1132.22,1252.761330
7,20,3585.75,6768.394739
8,23,10280.20,8644.702800
9,24,6184.59,5900.054649


In [197]:
oof_nn = oof_nn.sort_values(by="id")
oof_nn.columns = ["id", "loss_pred_nn"]
oof_nn_2.columns = ["id", "loss_pred_nn_2"]

In [198]:
oof_nn.head()

,id,loss_pred_nn
180536,1,1772.254663
114129,2,1453.589978
64050,5,3784.165991
166274,10,1002.060864
179990,11,3207.998926


In [75]:
stack_ds = oof_xgb.merge(oof_nn)

In [78]:
stack_ds = stack_ds.merge(oof_nn_2).set_index("id")

In [110]:
stack_ds["avg_nn_loss"] = 1.01*(0.58*stack_ds.loss_pred_nn + 0.42*stack_ds.loss_pred_nn_2)
stack_ds["loss_xgb_scales"] = 1.01*stack_ds.loss_pred_xgb

In [111]:
stack_ds.head()

,loss,loss_pred_xgb,loss_pred_nn,loss_pred_nn_2,avg_nn_loss,loss_xgb_scales
id,,,,,,
1,2213.18,1958.702840,1772.254663,1930.521753,1857.114109,1978.289868
2,1283.60,1766.157787,1453.589978,1461.915234,1471.657452,1783.819365
5,3005.09,4465.905258,3784.165991,3922.386230,3880.640677,4510.564310
10,939.85,1054.908671,1002.060864,999.840552,1011.139616,1065.457758
11,2763.85,3282.222525,3207.998926,3327.151514,3290.623443,3315.044750


In [219]:
mae(stack_ds.loss, stack_ds.loss_xgb_scales - 20)

1127.2263025583959

In [ ]:
1131.6166111766595 1132.6395716646759 1130.5996584028317(0.5 0.5) 1130.5528813394494(0.6 0.4) 1130.5513796498876(0.58 0.42)

In [103]:
for k in np.linspace(1,1.1,101):
    stack_ds["avg_nn_loss"] = k*(0.58*stack_ds.loss_pred_nn + 0.42*stack_ds.loss_pred_nn_2)
    print k, mae(stack_ds.loss, stack_ds.avg_nn_loss)

1.0 1130.55137965
1.001 1130.50553999
1.002 1130.46445635
1.003 1130.42871885
1.004 1130.39804177
1.005 1130.37202069
1.006 1130.3504744
1.007 1130.33437788
1.008 1130.32335512
1.009 1130.31738101
1.01 1130.3156728
1.011 1130.31891289
1.012 1130.32642883
1.013 1130.33895236
1.014 1130.35676421
1.015 1130.3802945
1.016 1130.40878792
1.017 1130.44263283
1.018 1130.48047558
1.019 1130.52253427
1.02 1130.56863156
1.021 1130.61916893
1.022 1130.67485832
1.023 1130.73541343
1.024 1130.80068202
1.025 1130.87027429
1.026 1130.94446088
1.027 1131.02318571
1.028 1131.10659525
1.029 1131.19458902
1.03 1131.28660358
1.031 1131.38323111
1.032 1131.48494054
1.033 1131.59200731
1.034 1131.70394093
1.035 1131.82079212
1.036 1131.94240763
1.037 1132.06855461
1.038 1132.1996142
1.039 1132.33557685
1.04 1132.47676416
1.041 1132.62301332
1.042 1132.77440054
1.043 1132.93014346
1.044 1133.09019716
1.045 1133.254759
1.046 1133.42441769
1.047 1133.59833009
1.048 1133.7766701
1.049 1133.96037808
1.05 1134.148

In [109]:
for k in np.linspace(1,1.1,101):
    print k, mae(stack_ds.loss, k*stack_ds.loss_pred_xgb)

1.0 1127.61001034
1.001 1127.56747049
1.002 1127.5297269
1.003 1127.49669178
1.004 1127.46809324
1.005 1127.44387427
1.006 1127.42402701
1.007 1127.4087273
1.008 1127.39820656
1.009 1127.39198388
1.01 1127.39013349
1.011 1127.39325616
1.012 1127.40101545
1.013 1127.41381401
1.014 1127.43130005
1.015 1127.45369285
1.016 1127.48094036
1.017 1127.51323279
1.018 1127.5501109
1.019 1127.59172019
1.02 1127.63794107
1.021 1127.68878887
1.022 1127.74428203
1.023 1127.80436448
1.024 1127.87020005
1.025 1127.94109595
1.026 1128.01719929
1.027 1128.09855901
1.028 1128.18488907
1.029 1128.27595495
1.03 1128.37144543
1.031 1128.471638
1.032 1128.57667678
1.033 1128.68546635
1.034 1128.79927229
1.035 1128.91783941
1.036 1129.04116695
1.037 1129.16889113
1.038 1129.30085078
1.039 1129.43726321
1.04 1129.5786707
1.041 1129.72449763
1.042 1129.87512726
1.043 1130.03024937
1.044 1130.18958402
1.045 1130.35338293
1.046 1130.52236599
1.047 1130.69604776
1.048 1130.87442111
1.049 1131.05757017
1.05 1131.24

In [114]:
y = stack_ds.loss
X = stack_ds[["loss_xgb_scales", "avg_nn_loss"]]

In [115]:
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.2, random_state=42)

In [117]:
X_train.head()

,loss_xgb_scales,avg_nn_loss
id,,
244665,1515.389025,1562.873842
555753,1226.463794,1525.698960
62502,5052.523655,6725.325991
149698,1308.177121,1249.677277
253191,1314.974355,1336.655413


In [121]:
score = {}
for w in np.linspace(0.4,0.7,301):
    y_pred = w*X_train.loss_xgb_scales + (1-w)*X_train.avg_nn_loss
    score[w] = mae(y_train, y_pred)

In [122]:
np.min(score.values())   # 1122.9085829257458 without scaling

1122.1808969338178

In [123]:
score

{0.40000000000000002: 1123.0203408854074,
 0.40100000000000002: 1123.0094049136214,
 0.40200000000000002: 1122.998552964654,
 0.40300000000000002: 1122.9878193745405,
 0.40400000000000003: 1122.9771332109908,
 0.40500000000000003: 1122.9665317719232,
 0.40600000000000003: 1122.9560021926859,
 0.40700000000000003: 1122.9455367763599,
 0.40800000000000003: 1122.9351795028722,
 0.40900000000000003: 1122.9248946667044,
 0.41000000000000003: 1122.914711671222,
 0.41100000000000003: 1122.904617995777,
 0.41200000000000003: 1122.8945929521876,
 0.41300000000000003: 1122.8846305343664,
 0.41400000000000003: 1122.8747530640503,
 0.41500000000000004: 1122.8649177223326,
 0.41600000000000004: 1122.8551421150516,
 0.41700000000000004: 1122.8454326700128,
 0.41800000000000004: 1122.8358071519285,
 0.41900000000000004: 1122.8262994810527,
 0.42000000000000004: 1122.8168228475972,
 0.42100000000000004: 1122.8074088434498,
 0.42200000000000004: 1122.798070302385,
 0.42300000000000004: 1122.78875972340

In [124]:
w = 0.554
y_pred_hold = w*X_hold.loss_xgb_scales + (1-w)*X_hold.avg_nn_loss
mae(y_hold, y_pred_hold)   # 1111.9776387586799 without scaling

1111.1240938614296

1102.92558 on LB

for post_calibration

In [33]:
X["prediction"] = w*X.loss_pred_xgb + (1-w)*X.loss_pred_nn
X["loss"] = y

In [34]:
X.head()

,loss_pred_xgb,loss_pred_nn,train_iso,loss,prediction
id,,,,,
1,1958.702840,1772.254663,1878.530124,2213.18,1878.530124
2,1766.157787,1453.589978,1631.753629,1283.60,1631.753629
5,4465.905258,3784.165991,4172.757373,3005.09,4172.757373
10,1054.908671,1002.060864,1032.184114,939.85,1032.184114
11,3282.222525,3207.998926,3250.306377,2763.85,3250.306377


In [37]:
X[["prediction", "loss"]].reset_index().to_csv("iso_reg/trainsubmission.csv", index=False)

In [39]:
preds["prediction"] = w*preds.loss_xgb + (1-w)*preds.loss_nn

In [42]:
preds[["id", "prediction"]].to_csv("iso_reg/testsubmission.csv", index=False)

result on public_lb: 1105.41214

In [19]:
preds.head()

,id,loss_xgb,loss_nn
0,4,1512.877930,1571.079736
1,6,1948.610962,1802.400940
2,9,10234.997070,8795.713916
3,12,6416.453125,7165.508398
4,15,792.276978,779.901031


In [20]:
preds["y_pred_subm"] = w*preds.loss_xgb + (1-w)*preds.loss_nn

In [21]:
preds.head()

,id,loss_xgb,loss_nn,y_pred_subm
0,4,1512.877930,1571.079736,1537.904707
1,6,1948.610962,1802.400940,1885.740652
2,9,10234.997070,8795.713916,9616.105314
3,12,6416.453125,7165.508398,6738.546893
4,15,792.276978,779.901031,786.955321


In [104]:
res = preds[["id", "y_pred_subm"]]
res.columns = ["id", "loss"]
res.to_csv("../submissions/submission_weighted_057xgb_043nn.csv", index=False)


linreg on 2 level

In [126]:
from sklearn.linear_model import LinearRegression, Ridge
linreg = LinearRegression(fit_intercept=False, n_jobs=-1)
ridge = Ridge(alpha=2.0, fit_intercept=False)

In [157]:
for reg in range(0,1100000,100000):
    ridge = Ridge(alpha=reg, fit_intercept=False)
    print reg, -cross_val_score(ridge, X_train, y_train, scoring="neg_mean_absolute_error", cv=10).mean()

 0 1139.69608083
100000 1139.69604882
200000 1139.69601681
300000 1139.6959848
400000 1139.69595279
500000 1139.69592078
600000 1139.69588877
700000 1139.69585677
800000 1139.69582476
900000 1139.69579275
1000000 1139.69576074


In [125]:
cross_val_score(linreg, X_train, y_train, scoring="neg_mean_absolute_error", cv=5, n_jobs=-1).mean()

-1139.6928025250486

In [163]:
ridge = Ridge(alpha=1000000, fit_intercept=False)

ridge.fit(X_train, y_train)
y_ridge = ridge.predict(X_hold)

In [164]:
mae(y_hold, y_ridge)

1127.1169173484448

In [165]:
ridge.coef_

array([ 0.74647971,  0.3475607 ])

xgb 2 level

In [168]:
import xgboost as xgb

In [183]:
def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mae(np.exp(y)-200, np.exp(yhat)-200)

In [184]:
model = xgb.XGBRegressor(max_depth = 2, learning_rate = 0.1, n_estimators = 100, 
                         colsample_bytree=100, colsample_bylevel=100)

In [271]:
space ={
        'max_depth': 2,
        'gamma' : 0,
        'colsample_bytree': 1,
        'colsample_bylevel': 1,
        #'silent': 1,
        'subsample': 1,
        'learning_rate': 0.01,
        'objective': 'reg:linear',
        'min_child_weight': 500,
        'booster': 'gbtree',
        'nthreads': 24,
        'seed': 42
        }

In [286]:
d_train = xgb.DMatrix(X, label=y)

In [287]:
xgb.cv(space, d_train, num_boost_round=300, nfold=5, metrics="mae", verbose_eval=1, seed=42)

[0]	train-mae:3006.48+4.86129	test-mae:3006.48+19.5306
[1]	train-mae:2976.43+4.81309	test-mae:2976.42+19.4162
[2]	train-mae:2946.67+4.76522	test-mae:2946.66+19.2955
[3]	train-mae:2917.23+4.71704	test-mae:2917.21+19.1766
[4]	train-mae:2888.08+4.67127	test-mae:2888.07+19.074
[5]	train-mae:2859.24+4.62668	test-mae:2859.23+18.9608
[6]	train-mae:2830.71+4.58336	test-mae:2830.69+18.8699
[7]	train-mae:2802.48+4.54171	test-mae:2802.46+18.7778
[8]	train-mae:2774.55+4.49856	test-mae:2774.54+18.6893
[9]	train-mae:2746.94+4.45842	test-mae:2746.91+18.5961
[10]	train-mae:2719.64+4.4181	test-mae:2719.61+18.5096
[11]	train-mae:2692.65+4.37826	test-mae:2692.61+18.431
[12]	train-mae:2665.94+4.33799	test-mae:2665.92+18.3565
[13]	train-mae:2639.53+4.3016	test-mae:2639.51+18.2844
[14]	train-mae:2613.41+4.2602	test-mae:2613.4+18.1969
[15]	train-mae:2587.58+4.22199	test-mae:2587.57+18.1227
[16]	train-mae:2562.04+4.18525	test-mae:2562.03+18.0409
[17]	train-mae:2536.79+4.1469	test-mae:2536.78+17.9665
[18]	trai

,test-mae-mean,test-mae-std,train-mae-mean,train-mae-std
0,3006.483887,19.530634,3006.483740,4.861285
1,2976.417773,19.416244,2976.425293,4.813088
2,2946.656006,19.295540,2946.673340,4.765223
3,2917.214502,19.176589,2917.226123,4.717039
4,2888.071289,19.074035,2888.082666,4.671272
5,2859.232324,18.960838,2859.244727,4.626685
6,2830.690576,18.869895,2830.708447,4.583358
7,2802.460254,18.777807,2802.479102,4.541708
8,2774.535351,18.689278,2774.553125,4.498556
9,2746.913330,18.596098,2746.939746,4.458423


In [ ]:

[254]	train-mae:1138.28+2.12741	test-mae:1140.84+8.28588


In [288]:
model = xgb.XGBRegressor(max_depth=2, colsample_bylevel=1, colsample_bytree=1, subsample=1, 
                        learning_rate=0.01, n_estimators = 260,)

In [289]:
model

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=260, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [290]:
model.fit(X_train, y_train, eval_metric = "mae")

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=260, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [291]:
mae(y_hold, model.predict(X_hold))

1129.6238443557052

1122.42307 - public_lb

all data xgb 2 level 

In [301]:
preds = preds.drop("y_pred_subm", 1)

ValueError: labels ['y_pred_subm'] not contained in axis

In [302]:
X_xgb = preds[["loss_xgb", "loss_nn"]]

In [305]:
model.fit(X.values, y, eval_metric = "mae")

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=260, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [306]:
y_pred_xgb_2_level = model.predict(X_xgb.values)

In [307]:
y_pred_xgb_2_level

array([ 1628.70666504,  2060.79541016,  8915.28515625, ...,  2538.61425781,
        1202.11120605,  3092.0402832 ], dtype=float32)

In [308]:
preds["y_pred_xgb_2_level"] = y_pred_xgb_2_level
preds.head(10)

,id,loss_xgb,loss_nn,y_pred_xgb_2_level
0,4,1512.877930,1571.079736,1628.706665
1,6,1948.610962,1802.400940,2060.795410
2,9,10234.997070,8795.713916,8915.285156
3,12,6416.453125,7165.508398,6784.024902
4,15,792.276978,779.901031,1202.111206
5,17,2470.061768,2339.430933,2538.614258
6,21,2181.214355,2055.929663,2269.303711
7,28,817.299500,763.764325,1202.111206
8,32,2478.701172,3058.344263,2750.553223
9,43,3164.730713,2949.261353,3245.229492


In [309]:
res = preds[["id", "y_pred_xgb_2_level"]]
res.columns = ["id", "loss"]
res.to_csv("../submissions/submission_xgb_2_level.csv", index=False)